In [16]:
import  pyspark
from pyspark.sql  import  SparkSession
spark = SparkSession.builder \
    .appName("Read CSV Example") \
    .getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.functions import regexp_replace,col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [7]:
df=spark.read.load("G:\SPARK_PROJECTS\Google Play Store Analyis\googleplaystore.csv",format="csv",sep=",",escape='"',header=True,inferSchema=True)
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [9]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [18]:
df=df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
.withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
.withColumn("Price",col("Price").cast(IntegerType()))

df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [19]:
df.show(5)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|   10000|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|  500000|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5000000|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|

In [20]:
df.createOrReplaceTempView("googleplaystore")

In [26]:
spark.sql("SELECT * FROM googleplaystore")

DataFrame[App: string, Category: string, Rating: double, Reviews: int, Size: string, Installs: int, Type: string, Price: int, Content Rating: string, Genres: string, Last Updated: string, Current Ver: string, Android Ver: string]

In [28]:
#TOP 10 APPS BY Reviews
spark.sql("select *,googleplaystore.Reviews from googleplaystore order by Reviews desc limit 10").show()

+--------------------+-------------+------+--------+------------------+----------+----+-----+--------------+-------------+--------------+------------------+------------------+--------+
|                 App|     Category|Rating| Reviews|              Size|  Installs|Type|Price|Content Rating|       Genres|  Last Updated|       Current Ver|       Android Ver| Reviews|
+--------------------+-------------+------+--------+------------------+----------+----+-----+--------------+-------------+--------------+------------------+------------------+--------+
|            Facebook|       SOCIAL|   4.1|78158306|Varies with device|1000000000|Free|    0|          Teen|       Social|August 3, 2018|Varies with device|Varies with device|78158306|
|            Facebook|       SOCIAL|   4.1|78128208|Varies with device|1000000000|Free|    0|          Teen|       Social|August 3, 2018|Varies with device|Varies with device|78128208|
|  WhatsApp Messenger|COMMUNICATION|   4.4|69119316|Varies with device|1000

In [31]:
#Top 10 Apps by Installs
spark.sql("Select * ,googleplaystore.Installs from googleplaystore order by Installs desc limit 10").show()

+--------------------+-------------------+------+--------+------------------+----------+----+-----+--------------+-----------------+--------------+------------------+------------------+----------+
|                 App|           Category|Rating| Reviews|              Size|  Installs|Type|Price|Content Rating|           Genres|  Last Updated|       Current Ver|       Android Ver|  Installs|
+--------------------+-------------------+------+--------+------------------+----------+----+-----+--------------+-----------------+--------------+------------------+------------------+----------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|Varies with device|1000000000|Free|    0|      Everyone|    Communication| July 21, 2018|Varies with device|Varies with device|1000000000|
|Google Chrome: Fa...|      COMMUNICATION|   4.3| 9643041|Varies with device|1000000000|Free|    0|      Everyone|    Communication|August 1, 2018|Varies with device|Varies with device|1000000000|
|   Google Play

In [ ]:
select category SUM(Installs) as total_installs from googleplaystore group by category order by total_installs desc

In [36]:
#Grouping by distribution of apps
spark.sql("select category, SUM(Installs) as total_installs from googleplaystore group by category order by total_installs desc").show()

+-------------------+--------------+
|           category|total_installs|
+-------------------+--------------+
|               GAME|   35086024415|
|      COMMUNICATION|   32647276251|
|       PRODUCTIVITY|   14176091369|
|             SOCIAL|   14069867902|
|              TOOLS|   11452771915|
|             FAMILY|   10258263505|
|        PHOTOGRAPHY|   10088247655|
| NEWS_AND_MAGAZINES|    7496317760|
|   TRAVEL_AND_LOCAL|    6868887146|
|      VIDEO_PLAYERS|    6222002720|
|           SHOPPING|    3247848785|
|      ENTERTAINMENT|    2869160000|
|    PERSONALIZATION|    2325494782|
|BOOKS_AND_REFERENCE|    1921469576|
|             SPORTS|    1751174498|
| HEALTH_AND_FITNESS|    1583072512|
|           BUSINESS|    1001914865|
|            FINANCE|     876648734|
|          EDUCATION|     871452000|
|MAPS_AND_NAVIGATION|     724281890|
+-------------------+--------------+
only showing top 20 rows

